In [7]:
#Import package for data analysis
import sys
from pandas import Series,DataFrame,read_csv
import os
import pandas as pd
import numpy
from scipy.interpolate import UnivariateSpline,interp1d

In [2]:
model             = os.getcwd() #get current directory
finalhhdataframes = model+'/finalhhdataframes/' #get the final depository directory
list_csv=os.listdir(finalhhdataframes) #list file name in the raw data directory
farmer=pd.DataFrame()

In [8]:
def perc_with_spline(data, wt, percentiles):
    assert numpy.greater_equal(percentiles, 0.0).all(), "Percentiles less than zero" 
    assert numpy.less_equal(percentiles, 1.0).all(), "Percentiles greater than one" 
    data = numpy.asarray(data) 
    assert len(data.shape) == 1 
    if wt is None: 
        wt = numpy.ones(data.shape, numpy.float) 
    else: 
        wt = numpy.asarray(wt, numpy.float) 
        assert wt.shape == data.shape 
        assert numpy.greater_equal(wt, 0.0).all(), "Not all weights are non-negative." 
    assert len(wt.shape) == 1 
    i = numpy.argsort(data) 
    sd = numpy.take(data, i, axis=0)
    sw = numpy.take(wt, i, axis=0) 
    aw = numpy.add.accumulate(sw) 
    if not aw[-1] > 0: 
     raise ValueError("Nonpositive weight sum" )
    w = (aw)/aw[-1] 
    # f = UnivariateSpline(w,sd,k=1)
    f = interp1d(numpy.append([0],w),numpy.append([0],sd))
    return f(percentiles)


In [11]:
dec[0]

55.418897066493031

In [13]:
df.columns

Index(['idh', 'Y', 'totY', 'children', 'old', 'decile', 'cat1workers',
       'cat2workers', 'cat3workers', 'cat4workers', 'cat5workers',
       'cat6workers', 'cat7workers', 'nbpeople', 'weight'],
      dtype='object')

In [17]:
df.weight.describe()

count      1157.000000
mean       7165.117996
std       20397.102639
min           0.000000
25%        1013.635853
50%        1971.011951
75%        5143.247116
max      311468.625552
Name: weight, dtype: float64

In [22]:
for filename in list_csv: 
    df=read_csv(finalhhdataframes+filename)
    df = df[df.weight>0]
    dec = perc_with_spline(df.Y,df.weight*df.nbpeople,[0.2])
    df=df[df.Y<dec[0]]
    farmer.ix[filename,'farm']=((df.cat3workers)*df.weight).sum()+((df.cat4workers)*df.weight).sum()
    farmer.ix[filename,'popu']=sum(df[['cat1workers','cat2workers', 'cat3workers', 'cat4workers', 'cat5workers',
       'cat6workers']].sum(axis=1)*df.weight)

In [23]:
farmer['ratio'] = farmer.farm/farmer.popu

In [24]:
farmer.ratio.describe()

count    95.000000
mean      0.585933
std       0.275336
min       0.001205
25%       0.371082
50%       0.650474
75%       0.801366
max       0.994117
Name: ratio, dtype: float64

In [25]:
print(farmer.farm.sum()/farmer.popu.sum())

0.595185124902


In [26]:
farmer.to_csv("stephane_farmer.csv",encoding='utf-8',index=False)